<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 5
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

1. Algorithme de Lesk simplifié.
1. Utilisation de word2vec.
1. Classification supervisée (cours 9) utilisant des traits lexicaux :
  1. les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest* ;
  1. apparition de mots indicateurs dans le voisinage de *interest*.

Les méthodes (1) et (2) n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La méthode (3) vise à classifier les occurrences de *interest*, les sens étant les classes, en utilisant comme traits les mots du contexte (ce sera de l'apprentissage supervisé).

## 0. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html).  Il se trouve en bas de cette page.  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez brièvement répondre aux questions suivantes :

1. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?
2. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?  Est-ce qu'om considère le pluriel *interests* aussi ?  Que se passe-t-il si une phrase contient plusieurs occurrences du mot ?

In [13]:
# 1. Le fichier zip: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz et le fichier README: https://www.d.umn.edu/~tpederse/Data/README.int.txt
# 2. Fichier texte avec des phrases séparée par des $$ et les mots interest/s avec un underscore et un id pour désigner leur sens. Le pluriel est considéré. Une seule occurence est traitée par phrase. Si la phrase comporte plusieurs occurence, la phrase sera copié et la prochaine occurence sera traîtée. Les occurences non-traitée sont précédées d'une *

3. D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

In [14]:
# Sense 1 =  361 occurrences (15%) - readiness to give attention
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
# Sense 5 =  500 occurrences (21%) - a share in a company or business
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money

4. De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [15]:
# Le dictionnaire est LDOCE (Longman Dictionary of Contemporary English)
# Sense 1 = "something you want to know or learn more about"
# Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
# Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
# Sense 4 = "the things that bring advantages to someone or something"
# Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
# Sense 6 = "the extra money that you must pay back when you borrow money"

5. En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [16]:
# Veuillez répondre ici (en commentaire) à la question.
# Il y a 7 synsets présents sur WordNet.
# Sense 1 = "a sense of concern with and curiosity about someone or something"
# Sense 2 = "the power of attracting or holding one's attention (because it is unusual or exciting etc.)"
# Sense 3 = "a diversion that occupies one's time and thoughts (usually pleasantly)"
# Sense 4 = "a reason for wanting something done"
# Sense 5 = "a right or legal share of something; a financial involvement with something"
# Sense 6 = "a fixed charge for borrowing money; usually a percentage of the amount borrowed"

6. Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [17]:
import nltk
import numpy as np

In [18]:
from random import randrange

In [19]:
readme_definitions = [
"readiness to give attention",
"quality of causing attention to be given to",
"activity, etc. that one gives attention to",
"advantage, advancement or favor",
"a share in a company or business",
"money paid for the use of money"
]
dictionnary_definitions = [
"something you want to know or learn more about",
"a quality or feature of something that attracts your attention or makes you want to know more about it",
"an activity that you enjoy doing or a subject that you enjoy studying",
"the things that bring advantages to someone or something",
"if you have an interest in a particular company or industry, you own shares in it",
"the extra money that you must pay back when you borrow money"
]
synSet_definitions = [
"a sense of concern with and curiosity about someone or something",
"the power of attracting or holding one's attention (because it is unusual or exciting etc.)",
"a diversion that occupies one's time and thoughts (usually pleasantly)",
"a reason for wanting something done",
"a right or legal share of something; a financial involvement with something",
"a fixed charge for borrowing money; usually a percentage of the amount borrowed"
]

In [20]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).

senses1 = [[word for word in definition.split() if len(word) > 3] for definition in readme_definitions]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity,', 'etc.', 'that', 'gives', 'attention'], ['advantage,', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid', 'money']]


7. En combinant les définitions obtenues aux points (3) et (4) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [21]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = np.array([readme_definitions[i] + ' ' + dictionnary_definitions[i] for i in range(6)])

senses2 = [[word for word in np.unique(sentence.split()) if len(word) > 3 and "interest" not in word] for sentence in senses2]


print(senses2)

[['about', 'attention', 'give', 'know', 'learn', 'more', 'readiness', 'something', 'want'], ['about', 'attention', 'attracts', 'causing', 'feature', 'given', 'know', 'makes', 'more', 'quality', 'something', 'that', 'want', 'your'], ['activity', 'activity,', 'attention', 'doing', 'enjoy', 'etc.', 'gives', 'studying', 'subject', 'that'], ['advancement', 'advantage,', 'advantages', 'bring', 'favor', 'someone', 'something', 'that', 'things'], ['business', 'company', 'have', 'industry,', 'particular', 'share', 'shares'], ['back', 'borrow', 'extra', 'money', 'must', 'paid', 'that', 'when']]


8. Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Les phrases sont-elles déjà tokenisées ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [22]:
# Veuillez répondre ici à la question.
#Les phrases sont déjà tokenisées
import re
sentences = []
with open("interest-original.txt", 'r') as f:
  current_line = ""
  for line in f.readlines():
      if line == "$$\n":
        sentences.append(re.sub("  ", " ", current_line).split())
        current_line = ""
      else:
        current_line += re.sub(r"[^\w ]*", "", line)


print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])


Il y a 2368 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', 'has', 'nt', 'stalled', 'at', 'all', 'mr', 'hines', 'maintains'], ['it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr', 'gorbachev', 'succeed'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', 'expression', 'of', 'interest_1', 'because', 'under', 'chapter', '11', 'revco', 'has', 'exclusivity', 'rights', 'until', 'feb', '28']]


## 1. Algorithme de Lesk simplifié

Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [23]:
# Veuillez répondre ici à la question.
from nltk.wsd import lesk
import random

def get_index_of_regexp_match(regexp, words):
  for index, value in enumerate(words):
    if re.match(regexp, value):
      return index
  return -1

def get_index_sens(words):
  return get_index_of_regexp_match(r"^.*_\d$", words)

def wsd_lesk(senses, sentence, window_size = 100):
  word_index = get_index_sens(sentence)
  is_plural = "interests" in sentence[word_index]
  start = max(word_index - window_size, 0)
  end = min(word_index + window_size + 1, len(sentence))
  windowed_sentence = sentence[start:end]

  maximum_count = -1
  senses_with_maximum_count = []
  for index, sens in enumerate(senses):
    count = len(list(word for word in sens if word in windowed_sentence))
    if count > maximum_count:
      maximum_count = count
      senses_with_maximum_count = [index]
    elif count == maximum_count:
      senses_with_maximum_count.append(index)
  random.shuffle(senses_with_maximum_count)
  return senses_with_maximum_count[0] + 1

print(wsd_lesk(senses2, sentences[149]))

2


Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [40]:
# Veuillez répondre ici à la question.
def evaluate_wsd(fct_name, senses, sentences, window_size = 30):
  if fct_name == "wsd_lesk":
    score = 0
    for sentence in sentences:
      index = get_index_sens(sentence)
      sens_nb = int(sentence[index][-1])
      if sens_nb == wsd_lesk(senses, sentence, window_size= window_size):
        score += 1
    return score * 100 / len(sentences)
  elif fct_name == "wsd_word2vec":
    score = 0
    for sentence in sentences:
      index = get_index_sens(sentence)
      sens_nb = int(sentence[index][-1])
      if sens_nb == wsd_word2vec(senses, sentence, window_size= window_size):
        score += 1
    return score * 100 / len(sentences)
  else:
    print("Function {} not implemented".format(fct_name))

def best_score(fct_name, max_window_size):
  best_score = 0
  best_window = 0
  best_sens = 0
  for window_size in range(1, max_window_size):
    score = 0
    for i in range(3):
      score += evaluate_wsd(fct_name, senses2, sentences, window_size= window_size)
    score /= 3
    if score > best_score:
      best_score = score
      best_window = window_size
      best_sens = 2
    score = 0
    for i in range(10):
      score += evaluate_wsd(fct_name, senses1, sentences, window_size= window_size)
    score /= 10
    if score > best_score:
      best_score = score
      best_window = window_size
      best_sens = 1
  return (best_score, best_sens, best_window)
lesk_score = best_score("wsd_lesk", 30)
print("Le meilleur score de Lesk est {} avec le sens {} pour un voisinnage de {}".format(lesk_score[0], lesk_score[1], lesk_score[2]))

Le meilleur score de Lesk est 23.507882882882882 avec le sens 2 pour un voisinnage de 21


En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [25]:
# Le meilleur score est de 23%, avec senses2 et un voisinnage de 21. On suppose que la taille du voisinnage vient
# du peu de mots présents dans les senses, et il est très probable qu'aucun mot en commun ne soit trouvé avec aucun des 6 sens,
# et donc un sens au hasard est choisit. Augmenter le voisinage augmente les chances d'avoir un mot commun
# A noter que les scores varie, puisque les sens données sont aléatoires en cas d'égalité

## 2. Utilisation de word2vec pour la similarité contexte vs. synset

En réutilisant une partie du code de `wsd_lesk`, définissez maintenant une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec**.  On vous encourage à chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.

Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.  La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*. Vous pouvez choisir la taille de ce voisinage (`window_size`).  En cas d'égalité, tirer le sens au sort.

In [26]:
import gensim
from gensim.models import KeyedVectors
import gensim.downloader

word2vec_gfile = gensim.downloader.load("word2vec-google-news-300")
vector = word2vec_gfile.wv
vector.save("word2vec-google-news-300.gz")

# path_to_model = "word2vec-google-news-300.gz"
# vector = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

[==================================================] 100.0% 1662.8/1662.8MB downloaded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [27]:
# Veuillez répondre ici à la question.
def extract_window_words(sentence, window_size):
  word_index = get_index_sens(sentence)
  start = max(word_index - window_size, 0)
  end = min(word_index + window_size + 1, len(sentence))
  return sentence[start:end]

def wsd_word2vec(senses, sentence, window_size = 100):
  word_index = get_index_sens(sentence)
  is_plural = "interests" in sentence[word_index]
  windowed_sentence = extract_window_words(sentence, window_size)

  maximum_count = -1
  senses_with_maximum_count = []
  windowed_sentence = list(filter(lambda x: x in vector.vocab, windowed_sentence))
  for index, sens in enumerate(senses):
    sens = list(filter(lambda x: x in vector.vocab, sens))
    if len(sens) == 0 or len(windowed_sentence) == 0:
      count = 0
    else:
      count = vector.n_similarity(sens,windowed_sentence)
    if count > maximum_count:
      maximum_count = count
      senses_with_maximum_count = [index]
    elif count == maximum_count:
      senses_with_maximum_count.append(index)
  random.shuffle(senses_with_maximum_count)

  return senses_with_maximum_count[0] + 1

word2vec_score = best_score("wsd_word2vec", 30)
print("Le meilleur score de Word2Vec est {} avec le sens {} pour un voisinnage de {}".format(word2vec_score[0], word2vec_score[1], word2vec_score[2]))

Le meilleur score de Word2Vec est 45.777027027027025 avec le sens 2 pour un voisinnage de 12


Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec les précédents ?

In [41]:
# Le meilleur score est de 45%, avec senses2 et un voisinage de 12. 
# Les résultats sont meilleurs qu'avec notre algorithme de Lesk simplifié. Nous avons remarqué que certains mots
# des phrases n'appartiennent pas au vocabulaire, et qu'il est nécessaire de les trier.

## 3. Classification supervisée avec des traits lexicaux
Dans cette partie du labo, vous entraînerez des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits, pour créer les données en vue des expériences de classification.

Vous utiliserez le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  (Il existe de nombreux autres classifieurs supervisés, par exemple dans la boîte à outils `scikit-learn`.)

De plus, vous devrez séparer les 2368 occurrences en ensembles d'entraînement et de test.

### 3.A. Traits lexicaux positionnels

Dans cette première représentation des traits, vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou alors `NONE` si la fenêtre dépasse la limite de la phrase.  Vous ajouterez un trait qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devez donc générer une représentation formelle avec un dictionnaire Python suivi de l'index du sens :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
L'index du sens servira à l'entraînement, puis elle sera cachée à l'évaluation, et la prédiction du système sera comparée à elle pour dire si elle est correcte ou non.  Vous regrouperez toutes ces entrées dans une liste totale de 2368 éléments appelée `items_with_features_A`.

En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste, en vous aidant si nécessaire du livre NLTK.

In [29]:
# Veuillez répondre ici à la question.

def create_dict_around(sentence, index):
  end = len(sentence)
  [word,sense] = sentence[index].split("_")
  neighbours = {}
  neighbours['word-2'] = sentence[index - 2] if index >= 2  else 'NONE'
  neighbours['word-1'] = sentence[index - 1] if index >= 1  else 'NONE'
  neighbours['word0']  =  word
  neighbours['word+1'] = sentence[index + 1] if index < end - 1  else 'NONE'
  neighbours['word+2'] = sentence[index + 2] if index < end - 2  else 'NONE'
  return [neighbours , sense]

def extract_feature_A(sentence):
  index = get_index_sens(sentence)
  return create_dict_around(sentence, index)

items_with_features_A = [extract_feature_A(sent) for sent in sentences]
print(len(items_with_features_A))
print(items_with_features_A[151:154])

2368
[[{'word-2': 'NONE', 'word-1': 'investor', 'word0': 'interest', 'word+1': 'in', 'word+2': 'stock'}, '1'], [{'word-2': 'the', 'word-1': 'western', 'word0': 'interest', 'word+1': 'to', 'word+2': 'see'}, '4'], [{'word-2': 'expression', 'word-1': 'of', 'word0': 'interest', 'word+1': 'because', 'word+2': 'under'}, '1']]


On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Typiquement, on peut garder 80% des données pour l'entraînement et utiliser les 20% restants pour l'évaluation.  Veuillez faire cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ ("stratification"), et enregistrer les deux sous-ensembles de `items_with_features_A` sous les noms respectifs de `iwf_A_train` et `iwf_A_test`.

In [30]:
from random import shuffle
import math

In [31]:

# Veuillez répondre ici à la question.
def separate_list(the_list, train_rate= 0.8):
  index = math.floor(len(the_list) * train_rate)
  shuffle(the_list)
  return (the_list[:index], the_list[index:])


def separate_sentence_by_sens(items, nb_senses=6, indexed0 = False):
  separated = list([] for s in range(nb_senses))
  for item in items:
    sens_nb = int(item[1]) - 1
    separated[sens_nb].append(item)
  return separated

def make_train_test(items, train_rate= 0.8):
 train = []
 test = []
 separated = separate_sentence_by_sens(items)
 for sens in separated:
   (sens_train, sens_test) = separate_list(sens, train_rate = train_rate)
   train += sens_train
   test += sens_test
 random.shuffle(train)
 random.shuffle(test)
 return (train, test)

(iwf_A_train, iwf_A_test) = make_train_test(items_with_features_A)

print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

1891   477
[[{'word-2': 'bundesbank', 'word-1': 'raised', 'word0': 'interest', 'word+1': 'rates', 'word+2': 'NONE'}, '6'], [{'word-2': 'up', 'word-1': 'its', 'word0': 'interest', 'word+1': 'income', 'word+2': 'NONE'}, '6']] [[{'word-2': 'in', 'word-1': 'an', 'word0': 'interest', 'word+1': 'rate', 'word+2': 'swap'}, '6'], [{'word-2': '1988', 'word-1': 'including', 'word0': 'interest', 'word+1': 'on', 'word+2': 'past'}, '6']]


Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_train` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez (avec la fonction `accuracy`), et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs (voir la doc NLTK), et pouvez-vous expliquer cet affichage ?

In [32]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.
classifier_A = nltk.NaiveBayesClassifier.train(iwf_A_train)
print(nltk.classify.accuracy(classifier_A, iwf_A_test))

classifier_A.show_most_informative_features(15)

# Ce classifier est bien meilleur que les algorithmes précédents (88% contre 45% au mieux)
# Comme la fenêtre de voisinnage est ici très réduite (2), on peut trouver deux explications:
# La première est que l'algorithme de BayesNaif est bien meilleur que les prédédents,
# et que la position des mots dans le voisinnage apporte beaucoup d'information

# Les trains les plus informatifs représentent les traits qui permettent de mieux séparer 2 sens. Les informations affichées sont, dans l'ordre : 

# 1.   La position du trait et sa valeur
# 2.   Les sens que le trait permet de séparer
# 3.   La proportion de séparation du trait (sensA : sensB)


# Par exemple, on voit que la présence du mot "in" juste après le mot "interest" permet de séparer les sens 5 et 6, avec 73.8 plus de chance que le sens soit le n°5
# Il est intéressant de remarquer que le mot "interests" au pluriel sert énormément dans la séparation des sens 3 et 1.

# On remarque que certains sens sont plus faciles à séparer que d'autre : par exemple les sens 5 et 6 sont présent 6 fois dans les 15 traits les plus informatifs.
# Il serait intéressant d'avoir les traits les plus informatifs pour chaque couple de sens.

0.8888888888888888
Most Informative Features
                   word0 = 'interests'         3 : 1      =     92.7 : 1.0
                  word+1 = 'in'                5 : 6      =     73.2 : 1.0
                  word-1 = 'other'             3 : 6      =     40.0 : 1.0
                  word+1 = 'of'                4 : 6      =     35.1 : 1.0
                  word-2 = 'a'                 5 : 4      =     25.9 : 1.0
                  word+2 = 'and'               6 : 5      =     21.6 : 1.0
                  word+2 = 'on'                6 : 5      =     18.7 : 1.0
                  word-2 = 'company'           5 : 6      =     17.7 : 1.0
                  word-2 = 'the'               4 : 3      =     17.4 : 1.0
                  word+2 = 'to'                6 : 1      =     14.9 : 1.0
                  word-2 = 'have'              1 : 6      =     13.5 : 1.0
                  word-1 = 'the'               4 : 5      =     13.3 : 1.0
                  word-2 = 'other'             3 : 6   

In [33]:
# Question supprimée


### 3.B. Présence de mots indicateurs

Une deuxième façon d'encoder les traits lexicaux est de constituer un vocabulaire avec les mots qui apparaissent dans le voisinage de *interest* et de définir ces mots comme traits.  Par conséquent, pour chaque occurrence de *interest*, vous allez extraire la valeur de ces traits sous la forme :
```
[{('rate' : True), ('in' : False), ...}, 1]
```
où *'rate'*, *'in'* sont les mots du vocabulaire, True/False indiquent leur présence/absence autour de l'occurrence de *interest* qui est décrite, et le dernier nombre est le sens, entre 1 et 6.

Pour commencer, en partant de `sentences` et en fixant la taille de la fenêtre, veuillez constituer la liste de tous les mots observés autour de tous les voisinages de toutes les occurrences de *interest*.

In [34]:
word_list = []
# Veuillez répondre ici à la question.
windowed_sentences = list(extract_window_words(sentence, 10) for sentence in sentences)
for sentence in windowed_sentences:
  for word in sentence:
    word_list.append(word.split("_")[0])
print(len(word_list))
print(word_list[:50])

38583
['slide', 'amid', 'signs', 'that', 'portfolio', 'managers', 'expect', 'further', 'declines', 'in', 'interest', 'rates', 'longer', 'maturities', 'are', 'thought', 'to', 'indicate', 'declining', 'interest', 'rates', 'because', 'they', 'permit', 'portfolio', 'managers', 'to', 'retain', 'relatively', 'higher', 'before', 'they', 'blip', 'down', 'because', 'of', 'recent', 'rises', 'in', 'shortterm', 'interest', 'rates', 'vice', 'chairman', 'of', 'wr', 'grace', 'co', 'which', 'holds']


En utilisant un objet `nltk.FreqDist`, veuillez sélectioner les 500 mots les plus fréquents (vous pourrez aussi faire varier ce nombre), dans une liste appelée `vocabulary`.  À votre avis, est-ce une bonne idée d'enlever les *stopwords* de cette liste pour construire les traits ?

In [35]:
# Veuillez répondre ici à la question.
words_freq = nltk.FreqDist(list(word for word in word_list if len(word) > 3))
vocabulary = words_freq.most_common(500)
print(vocabulary[:50])

# Oui c'est une bonne idée : les stopwords seront probablement présent dans les mêmes proportions dans tous les sens du mot. 
# Ils prennent donc la place de mots plus spécifiques

[('interest', 2066), ('rates', 652), ('that', 418), ('interests', 357), ('said', 231), ('million', 210), ('from', 200), ('with', 194), ('will', 183), ('rate', 167), ('company', 154), ('have', 154), ('which', 140), ('would', 133), ('bonds', 122), ('payments', 120), ('lower', 114), ('about', 94), ('they', 87), ('debt', 87), ('other', 87), ('also', 86), ('their', 85), ('market', 81), ('high', 80), ('because', 77), ('higher', 75), ('than', 74), ('after', 73), ('federal', 71), ('bank', 71), ('more', 70), ('short', 70), ('this', 64), ('billion', 64), ('foreign', 62), ('income', 60), ('some', 59), ('dollar', 57), ('been', 57), ('year', 56), ('stock', 55), ('general', 54), ('were', 54), ('annual', 53), ('says', 53), ('1989', 52), ('banks', 51), ('shares', 49), ('group', 49)]


Veuillez maintenant créer l'ensemble total de données formatées, en convertissant chaque phrase contenant une occurrence de *interest* à un dictionnaire de traits/valeurs (suivi du numéro du sens), comme exemplifié au début de cette section 3B.  Cet ensemble sera appelé `items_with_features_B`.

In [43]:
items_with_features_B = []
# Veuillez répondre ici à la question.
def extract_feature_b(sentences, window_size, vocabulary):
  featured_items = []
  windowed_sentences = list(extract_window_words(sentence, window_size) for sentence in sentences)
  for sentence in windowed_sentences:
    traits = {}
    index_word = get_index_sens(sentence)
    [word_interest, sens_nb] = sentence[index_word].split("_")
    sentence[index_word] = word_interest
    for voc in vocabulary:
      voc_word = voc[0]
      traits[voc_word] = voc_word in sentence
    featured_items.append([traits, sens_nb])
  return featured_items

items_with_features_B = extract_feature_b(sentences, 15, vocabulary)

print(len(items_with_features_B))
print((items_with_features_B[50][:50]))


2368
[{'interest': False, 'rates': False, 'that': False, 'interests': True, 'said': True, 'million': True, 'from': False, 'with': False, 'will': False, 'rate': False, 'company': False, 'have': False, 'which': False, 'would': False, 'bonds': False, 'payments': False, 'lower': False, 'about': False, 'they': False, 'debt': False, 'other': False, 'also': False, 'their': False, 'market': False, 'high': False, 'because': False, 'higher': False, 'than': False, 'after': False, 'federal': False, 'bank': False, 'more': False, 'short': False, 'this': False, 'billion': False, 'foreign': False, 'income': False, 'some': False, 'dollar': False, 'been': False, 'year': False, 'stock': True, 'general': False, 'were': False, 'annual': False, 'says': False, '1989': False, 'banks': False, 'shares': True, 'group': False, 'minority': False, 'investors': False, 'corp': True, 'companies': False, 'loans': False, 'reserve': False, 'inflation': False, 'last': False, 'could': False, 'securities': False, 'buying': 

Comme dans la section 3A, veuillez créer maintenant deux sous-ensembles de `items_with_features_B` appelés `iwf_B_train` (80% des items) et `iwf_B_test` (20% des items), avec une sélection aléatoire mais stratifiée.

In [44]:

# Veuillez répondre ici à la question.
(iwf_B_train, iwf_B_test) = make_train_test(items_with_features_B)

print(len(iwf_B_train), ' ', len(iwf_B_test))

1891   477


Comme pour la section 3A, veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_B_train` et la tester sur `iwf_B_train`.  Veuillez obtenir le score de ce classifieur.  En expérimentant avec différentes largeurs de fenêtres et tailles du vocabulaire, quel est le meilleur score que vous obtenez, et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs ?

In [47]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.
classifier_B = nltk.NaiveBayesClassifier.train(iwf_B_train)
print(nltk.classify.accuracy(classifier_B, iwf_B_test))

classifier_B.show_most_informative_features(15)

# Ce classifier est légèremment moins bons que celui sur les traits positionnel,
# même pour des fenêtres beaucoup plus larges (ici 15).
# Il semblerait que la position des mots voisins soit plus à même d'indiquer la signification
# que la simple présence ou non d'un mot.

0.8574423480083857
Most Informative Features
                    best = True                4 : 6      =    109.8 : 1.0
                   rates = True                6 : 1      =     97.0 : 1.0
                  pursue = True                3 : 5      =     74.1 : 1.0
               interests = True                3 : 1      =     70.9 : 1.0
                interest = False               3 : 1      =     70.9 : 1.0
                building = True                2 : 6      =     66.8 : 1.0
                  soviet = True                2 : 6      =     66.8 : 1.0
                personal = True                3 : 6      =     56.7 : 1.0
               expressed = True                1 : 6      =     52.0 : 1.0
                national = True                2 : 6      =     47.7 : 1.0
                official = True                2 : 6      =     47.7 : 1.0
                congress = True                2 : 6      =     47.7 : 1.0
                    deal = True                2 : 6   

In [42]:
# Veuillez recopier ici en conclusion les scores des quatre 
# expériences, pour pouvoir les comparer d'un coup d'oeil.
# Algorithme de Lesk             : 23%
# Algorithme word2vec            : 45%
# Classifier traits positionnels : 88%
# Classifier mots indicateurs    : 85%


## Fin du laboratoire

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.